<a href="https://colab.research.google.com/github/diwakar-vsingh/Fluent-Python/blob/version-0.0/02-array-seq/02_array_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Built-In Sequences

The standard library offers a rich selection of sequence types implemented in C:

***Container Sequences***:
list, tuple, and collections.deque can hold items of different types.

***Flat Sequences***:
str, bytes, bytearray, memoryview, and array.array hold items of one type.

*Container sequences* hold references to the objects they contain, which may be of any type, while *flat sequences* physically store the value of each item within its own memory space, and not as distinct objects. Thus, flat sequences are more compact, but they are limited to holding primitive values like characters, bytes, and numbers.

Another way of grouping sequence types is by mutability:

***Mutable sequences***:
list, bytearray, array.array, collections.deque, and memoryview 

***Immutable sequences***:
tuple, str, and bytes

<img src="https://github.com/diwakar-vsingh/Fluent-Python/blob/version-0.0/02-array-seq/images/table1.png?raw=true" width="700px"/>

## List Comprehensions and Generator Expressions

A quick way to build a sequence is using a list comprehension (if the target is a list) or a generator expression (for all other kinds of sequences).

In [1]:
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols]
codes

[36, 162, 163, 165, 8364, 164]

### Listcomps Versus map and filter

ist comprehensions build lists from sequences or any other iterable type by filtering and transforming items. The `filter` and `map` built-ins can be composed to do the same, but readability suffers, as we will see next.

In [2]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
print(beyond_ascii)
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols))) 
print(beyond_ascii)

[162, 163, 165, 8364, 164]
[162, 163, 165, 8364, 164]


#### Speed test comparing listcomp with `filter/map`.

In [3]:
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.019 0.010 0.008
listcomp + func : 0.013 0.013 0.012
filter + lambda : 0.012 0.012 0.012
filter + func   : 0.012 0.012 0.012


### Cartesian Products

Listcomps can generate lists from the Cartesian product of two or more iterables. The items that make up the cartesian product are tuples made from items from every input iterable. The resulting list has a length equal to the lengths of the input iterables mul‐ tiplied.

For example, imagine you need to produce a list of T-shirts available in two colors and three sizes. The following code block shows how to produce that list using a listcomp. The result has six items.

In [4]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [5]:
# code snippet to be executed only once  
mysetup = """
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
"""

# code snippets whose execution time is to be measured
code1 = """
temp = [(color, size) for color in colors for size in sizes]
""" 

code2 = """
temp = [] 
for color in colors: 
  for size in sizes: 
    temp.append((color, size))
""" 

def clock(label, code):
    res = timeit.repeat(setup=mysetup, stmt=code, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))

clock('listcomp         :', code1)
clock('without listcomp :', code2)

listcomp         : 0.007 0.005 0.005
without listcomp : 0.006 0.008 0.006


### Generator Expression

To initialize tuples, arrays, and other types of sequences, you could also start from a listcomp, but a genexp saves memory because it yields items one by one using the iterator protocol instead of building a whole list just to feed another constructor.

Genexps use the same syntax as listcomps, but are enclosed in parentheses rather than brackets.

####  Initializing a tuple and an array from a generator expression

In [6]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [7]:
import array
array.array("I", (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

#### Genexp with cartesian product

The generator expression feeds the for loop producing one item at a time. 

In [8]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
  print(tshirt)

black S
black M
black L
white S
white M
white L


## Tuples are not just Immutable Lists

Tuples do double duty: they can be used as immutable lists and also as records with no field names.

### Tuples as Records

Tuples hold records: each item in the tuple holds the data for one field and the position of the item gives its meaning.

The following example shows tuples being used as records. Note that in every expression, sorting the tuple would destroy the information because the meaning of each data item is given by its position in the tuple

In [9]:
# Latitude and longitude of the Los Angeles International Airport.
lax_coordinates = (33.9425, -118.408056)

# Data about Tokyo: name, year, population (millions), population change (%), area (km2).
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)

# A list of tuples of the form (country_code, passport_number).
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

for passport in sorted(traveler_ids):
  print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


### Tuple Unpacking

The most visible form of tuple unpacking is parallel assignment; that is, assigning items from an iterable to a tuple of variables.

In [10]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates # tuple unpacking
print(latitude, longitude)

33.9425 -118.408056


An elegant application of tuple unpacking is swapping the values of variables without using a temporary variable:

`b, a = a, b`

Another example of tuple unpacking is prefixing an argument with a star when calling
a function:

In [11]:
divmod(20, 8)

(2, 4)

In [12]:
t = (20, 8)
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

Sometimes when we only care about certain parts of a tuple when unpacking, a dummy variable like _ is used as placeholder

In [13]:
import os

# os.path.split() function builds a tuple (path, last_part) from a filesystem path
_, filename = os.path.split('/home/luciano/.ssh/idrsa.pub')
filename

'idrsa.pub'

#### Using * to grab exceess items


In [14]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [15]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In the context of parallel assignment, the * prefix can be applied to exactly one variable, but it can appear in any position:

In [16]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

### Nested Tuple Unpacking

The tuple to receive an expression to unpack can have nested tuples, like (a, b, (c, d)), and Python will do the right thing if the expression matches the nesting structure. 


In [17]:
metro_areas = [('Tokyo', 'JP', 36.933, (35.689722,139.691667)), 
               ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)), 
               ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
               ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
               ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))]
print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.')) 
fmt = '{:15} | {:9.4f} | {:9.4f}'

for name, cc, pop, (latitude, longitude) in metro_areas: 
  if longitude <= 0: 
    print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


### Named Tuple

The collections.namedtuple function is a factory that produces subclasses of tuple enhanced with field names and a class name—which helps debugging.

In [18]:
import collections

City = collections.namedtuple("City", ["name", "country", "population", "coordinates"])
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [19]:
print(tokyo.population)
print(tokyo.coordinates)
print(tokyo[1])

36.933
(35.689722, 139.691667)
JP


A named tuple type has a few attributes in addition to those inherited from tuple. The `_fields` class attribute, the class method `_make(iterable)`, and the `_asdict()` instance method.

1. `_make()` allow you to instantiate a named tuple from an iterable: `City(*delhi_data)` would do the same.
2. `_asdict()` returns a collections.OrderedDict built from the named tuple
instance.

In [20]:
# Class attribute
City._fields

('name', 'country', 'population', 'coordinates')

In [21]:
LatLong = collections.namedtuple('LatLong', ["lat", "long"])
delhi_data = ("Delhi NCR", "IN", 21.935, LatLong(28.613889, 77.208889))
delhi = City._make(delhi_data) # Class Method
print(delhi)
print(City(*delhi_data))
print(delhi._asdict())

City(name='Delhi NCR', country='IN', population=21.935, coordinates=LatLong(lat=28.613889, long=77.208889))
City(name='Delhi NCR', country='IN', population=21.935, coordinates=LatLong(lat=28.613889, long=77.208889))
OrderedDict([('name', 'Delhi NCR'), ('country', 'IN'), ('population', 21.935), ('coordinates', LatLong(lat=28.613889, long=77.208889))])


In [22]:
for (key, value) in delhi._asdict().items():
  print(key + ":", value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613889, long=77.208889)


### Tuples as Immutable Lists

When using a tuple as an immutable variation of list, it helps to know how similar they actually are. As you can see in Table 2-1, tuple supports all list methods that do not involve adding or removing items, with one exception—tuple lacks the __re versed__ method. However, that is just for optimization; reversed(my_tuple) works without it.


<center><img src="https://github.com/diwakar-vsingh/Fluent-Python/blob/version-0.0/02-array-seq/images/table2.png?raw=true" width="600px"/></center>
<center><img src="https://github.com/diwakar-vsingh/Fluent-Python/blob/version-0.0/02-array-seq/images/table3.png?raw=true" width="600px"/></center>

## Slicing

A common feature of list, tuple, str, and all sequence types in Python is the support of slicing operations, which are more powerful than most people realize

### Why Slices and Range Exclude the Last Item

The Pythonic convention of excluding the last item in slices and ranges works well with the zero-based indexing used in Python, C, and many other languages.

In [23]:
l = [10, 20, 30, 40, 50, 60]
l[:2], l[2:]

([10, 20], [30, 40, 50, 60])

### Slice Object

This is no secret, but worth repeating just in case: `s[a:b:c]` can be used to specify a stride or step c, causing the resulting slice to skip items. The stride can also be negative, returning items in reverse. 

In [24]:
s = 'bicycle'
print(s[::3])
print(s[::-1])

bye
elcycib


The notation a:b:c is only valid within `[]` when used as the indexing or subscript operator, and it produces a slice object: `slice(a, b, c)`.

Knowing about slice objects is useful because it lets you assign names to slices

In [25]:
invoice = """
0.....6.................................40........52...55.......
1909 Pimoroni PiBrella                      $17.50
1489 6mm Tactile Switch x20                  $4.95
1510 Panavise Jr. - PV-201                  $28.00
1601 PiTFT Mini Kit 320x240                 $34.50
"""

In [26]:
SKU = slice(0, 6)
DESCRIPTION = slice(5, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:] 

for item in line_items:
  print(item[UNIT_PRICE], item[DESCRIPTION])

    $17.50 Pimoroni PiBrella                  
     $4.95 6mm Tactile Switch x20             
    $28.00 Panavise Jr. - PV-201              
    $34.50 PiTFT Mini Kit 320x240             
 


### Assigning to Slices

Mutable sequences can be grafted, excised, and otherwise modified in place using slice notation on the left side of an assignment statement or as the target of a `del` statement.

In [27]:
l = list(range(10))
print(l)
l[2:5] = [20, 30]
print(l)
del l[5:7]
print(l)
l[3::2] = [11, 22]
print(l)
l[2:5] = [100]
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 8, 9]
[0, 1, 20, 11, 5, 22, 9]
[0, 1, 100, 22, 9]


### Using + and * Sequences

Usually both operands of + must be of the same sequence type, and neither of them is modified but a new sequence of the same type is created as result of the concatenation. Both + and * always create a new object, and never change their operands.

To concatenate multiple copies of the same sequence, multiply it by an integer. Again, a new sequence is created:

In [28]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [29]:
5 * "abcd"

'abcdabcdabcdabcdabcd'

### Building lists of lists

Sometimes we need to initialize a list with a certain number of nested lists—for example, to distribute students in a list of teams or to represent squares on a game board. 

In [30]:
# A list with three lists of length 3 can represent a tic-tac-toe board
board = [['_'] * 3 for i in range(3)] 
print(board)
board[1][2] = "X"
print(board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]


In [31]:
# A list with three references to the same list is useless
weird_board = [['_'] * 3] * 3
print(weird_board)
weird_board[1][2] = 'O'
print(weird_board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', 'O'], ['_', '_', 'O'], ['_', '_', 'O']]


## Augmented Assignment with Sequences

The augmented assignment operators += and *= behave very differently depending on the first operand.


The special method that makes += work is `__iadd__` (for “in-place addition”). However, if `__iadd__` is not implemented, Python falls back to calling `__add__`.

If a implements `__iadd__`, that will be called. In the case of mutable sequences (e.g., list, bytearray, array.array), a will be changed in place (i.e., the effect will be similar to a.extend(b)). However, when a does not implement `__iadd__`, the expression a += b has the same effect as a = a + b: the expression a + b is evaluated first, producing a new object, which is then bound to a. In other words, the identity of the object bound to a may or may not change, depending on the availability of `__iadd__`.

In [32]:
l = [1, 2, 3]
print(id(l))
l *= 2
print(id(l))

139998951121032
139998951121032


In [33]:
t = (1, 2, 3)
print(id(t))
t *= 2
print(id(t))

139998951061040
139998975585544


Repeated concatenation of immutable sequences is inefficient, because instead of just appending new items, the interpreter has to copy the whole target sequence to create a new one with the new items concatenated.

## list.sort and the sorted Built-In Function

The `list.sort` method sorts a list in place—that is, without making a copy. It returns `None` to remind us that it changes the target object, and does not create a new list. This is an important Python API convention: functions or methods that change an object in place should return `None` to make it clear to the caller that the object itself was changed, and no new object was created. The same behavior can be seen, for example, in the `random.shuffle` 
function.

In contrast, the built-in function `sorted` creates a new list and returns it. In fact, it accepts any iterable object as an argument, including immutable sequences and generators. Regardless of the type of iterable given to `sorted`, it always returns a newly created list.

Both `list.sort` and `sorted` take two optional, keyword-only arguments:

`reverse`: If True, the items are returned in descending order (i.e., by reversing the comparison of the items). The default is False.

In [34]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
print("This produces a new list of strings sorted alphabetically.")
print(sorted(fruits))
print("\n")
print("Inspecting the original list")
print(fruits)
print("\n")
print("This is simply reverse alphabetical ordering.")
print(sorted(fruits, reverse=True))
print("\n")
print("A new list of strings, now sorted by length.")
print(sorted(fruits, key=len))
print("\n")
print("These are the strings sorted in descending order of length.")
print(sorted(fruits, key=len, reverse=True))
print("\n")
print("the ordering of the original fruits list has not changed")
print(fruits)
print("\n")
print("This sorts the list in place, and returns None ")
print(fruits.sort())
print("\n")
print("Now fruits is sorted.")
print(fruits)

This produces a new list of strings sorted alphabetically.
['apple', 'banana', 'grape', 'raspberry']


Inspecting the original list
['grape', 'raspberry', 'apple', 'banana']


This is simply reverse alphabetical ordering.
['raspberry', 'grape', 'banana', 'apple']


A new list of strings, now sorted by length.
['grape', 'apple', 'banana', 'raspberry']


These are the strings sorted in descending order of length.
['raspberry', 'banana', 'grape', 'apple']


the ordering of the original fruits list has not changed
['grape', 'raspberry', 'apple', 'banana']


This sorts the list in place, and returns None 
None


Now fruits is sorted.
['apple', 'banana', 'grape', 'raspberry']


In [45]:
sorted({1: 'D', 2: 'B', 3: 'B', 4: 'E', 5: 'A'})

[1, 2, 3, 4, 5]

### Key Functions

Both `list.sort()` and `sorted()` have a key parameter to specify a function (or other callable) to be called on each list element prior to making comparisons.

In [46]:
sorted("This is a test string from Andrew".split(), key=str.lower)

['a', 'Andrew', 'from', 'is', 'string', 'test', 'This']

The value of the key parameter should be a function (or other callable) that takes a single argument and returns a key to use for sorting purposes. This technique is fast because the key function is called exactly once for each input record.

A common pattern is to sort complex objects using some of the object’s indices as keys.

In [80]:
student_tuples = [('john', 'A', 15), 
                  ('jane', 'B', 12),
                  ('dave', 'B', 10)]
print(student_tuples)
print(sorted(student_tuples, key = lambda student: student[2]))

[('john', 'A', 15), ('jane', 'B', 12), ('dave', 'B', 10)]
[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]


The same technique works for objects with named attributes. 

In [55]:
class Student:
  def __init__(self, name, grade, age):
    self.name = name
    self.grade = grade
    self.age = age
  
  def __repr__(self):
    return repr((self.name, self.grade, self.age))

In [56]:
student_objects = [Student('john', 'A', 15), 
                  Student('jane', 'B', 12),
                  Student('dave', 'B', 10)]
print(student_objects)
print(sorted(student_objects, key = lambda student: student.age))   # sort by age

[('john', 'A', 15), ('jane', 'B', 12), ('dave', 'B', 10)]
[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]


In [61]:
Students = collections.namedtuple("Students", ["name", "grade", "age"])
students_tuples = [Students('john', 'A', 15), 
                  Students('jane', 'B', 12),
                  Students('dave', 'B', 10)]
print(students_tuples)
print(sorted(students_tuples, key = lambda student: student.age))

[Students(name='john', grade='A', age=15), Students(name='jane', grade='B', age=12), Students(name='dave', grade='B', age=10)]
[Students(name='dave', grade='B', age=10), Students(name='jane', grade='B', age=12), Students(name='john', grade='A', age=15)]


### Operator Module Functions

The key-function patterns shown above are very common, so Python provides convenience functions to make accessor functions easier and faster. The operator module has `itemgetter()`, `attrgetter()`, and a `methodcaller()` function.

In [62]:
from operator import itemgetter, attrgetter

In [65]:
print(sorted(student_tuples, key=itemgetter(2)))
print(sorted(student_objects, key=attrgetter('age')))
print(sorted(students_tuples, key=attrgetter('age')))
print(sorted(students_tuples, key=itemgetter(2)))

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]
[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]
[Students(name='dave', grade='B', age=10), Students(name='jane', grade='B', age=12), Students(name='john', grade='A', age=15)]
[Students(name='dave', grade='B', age=10), Students(name='jane', grade='B', age=12), Students(name='john', grade='A', age=15)]


The operator module functions allow multiple levels of sorting. For example, to sort by grade then by age:

In [68]:
print(sorted(student_tuples, key=itemgetter(1, 2)))
print(sorted(student_objects, key=attrgetter("grade", "age")))
print(sorted(students_tuples, key=attrgetter("grade", "age")))

[('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)]
[('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)]
[Students(name='john', grade='A', age=15), Students(name='dave', grade='B', age=10), Students(name='jane', grade='B', age=12)]


### Sort Stability and Complex Sorts

Sorts are guaranteed to be stable. That means that when multiple records have the same key, their original order is preserved.

In [69]:
data = [('red', 1), ('blue', 1), ('red', 2), ('blue', 2)]
sorted(data, key=itemgetter(0))

[('blue', 1), ('blue', 2), ('red', 1), ('red', 2)]

This wonderful property lets you build complex sorts in a series of sorting steps. For example, to sort the student data by descending grade and then ascending age, do the age sort first and then sort again using grade:

In [87]:
s = sorted(student_objects, key=attrgetter("age"))
print(s)
print(sorted(s, key=attrgetter("grade"), reverse=True))

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]
[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]


In [89]:
ss = sorted(student_objects, key=attrgetter("grade"), reverse=True)
sorted(ss, key=attrgetter("age"))

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

This can be abstracted out into a wrapper function that can take a list and tuples of field and order to sort them on multiple passes.

In [83]:
def multisort(xs, specs):
  for key, reverse in reversed(specs):
    xs.sort(key=attrgetter(key), reverse=reverse)
  return xs

In [86]:
multisort(student_objects, (('grade', True), ('age', False)))

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

## Deques and other Queues

The `.append` and `.pop` methods make a list usable as a stack or a queue. But inserting and removing from the left of a list (the 0-index end) is costly because the entire list must be shifted.

The class `collections.deque` is a thread-safe double-ended queue designed for fast inserting and removing from both ends. It is also the way to go if you need to keep a list of “last seen items” or something like that, because a deque can be bounded—i.e., created with a maximum length—and then, when it is full, it discards items from the opposite end when you append new ones. 


In [36]:
from collections import deque

In [44]:
# The optional maxlen argument sets the maximum number of items allowed in this instance of deque
dq = deque(range(10), maxlen=10)
print(dq)

# Rotating with n > 0 takes items from the right end and prepends them to the left; 
# when n < 0 items are taken from left and appended to the right.
dq.rotate(3)
print(dq)
dq.rotate(-4)
print(dq)

# Appending to a deque that is full (len(d) == d.maxlen) discards items from the other end;
dq.appendleft(-1)
print(dq)
dq.append(-1)
print(dq)
dq.extend([11, 22, 33])
print(dq)

# extendleft(iter) works by appending each successive item of the iter argument 
# to the left of the deque, therefore the final position of the items is reversed.
dq.extendleft([10, 20, 30, 40])
print(dq)

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)
deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, -1], maxlen=10)
deque([4, 5, 6, 7, 8, 9, -1, 11, 22, 33], maxlen=10)
deque([40, 30, 20, 10, 4, 5, 6, 7, 8, 9], maxlen=10)
